# Processamento dos dados do extrato da Vitreo para capturar informações sobre BTC (aluguel de ações)

In [ ]:
import pandas as pd

In [ ]:
def processa_descricao(row):
    saida = ""
    if "TAXA DE REMUNERAÇÃO" in row['Descrição']:
        saida = "TAXA DE REMUNERAÇÃO"
    elif "COMISSÃO BTC" in row['Descrição']:
        saida = "COMISSÃO BTC"
    elif "REEMB.DIVIDENDOS/JUROS" in row['Descrição']:
        saida = "REEMB.DIVIDENDOS"
    elif "TAXA NEGOCIACAO EMPRESTIMO - BTC" in row['Descrição']:
        saida = "TAXA NEGOCIACAO EMPRESTIMO"
    elif "TAXA DE PERMANÊNCIA OP. ESTRUTURADAS" in row['Descrição']:
        saida = "TAXA DE PERMANÊNCIA OP. ESTRUTURADAS"       
    else:
        saida = row['Descrição']
    
    return saida
        

In [ ]:
def calcula_custos_comissao(row):
    comissao = 0
    negociacao = 0
    reembolso = 0
    if row['FILTRO'] == 'TAXA DE REMUNERAÇÃO':
        comissao = row['COMISSÃO BTC'] * (row['Valor de transação'] / row['TAXA DE REMUNERAÇÃO'])
     
    return comissao

In [ ]:
# path do arquivo de extrato
# formato do arquivo
# Data Movimentação, Data Liquidação, Tipo (C/D), Descrição, Valor de transação, Saldo

ARQUIVO_EXTRATO = r'./extrato_vitreo.xlsx'

In [ ]:
dados = pd.read_excel(ARQUIVO_EXTRATO)
dados.drop(columns=['Saldo'], inplace=True)

In [ ]:
dados_btc = dados[(dados['Descrição'].str.contains('BTC')) | 
                  (dados['Descrição'].str.contains('EMPRESTIMO')) |
                  (dados['Descrição'].str.contains('TAXA')) |
                  (dados['Descrição'].str.contains('REEMB.DIVIDENDOS/JUROS'))
                 ]

#remove as linhas com o fundo BTCI11
dados_btc = dados_btc[(~dados_btc['Descrição'].str.contains('BTCI11'))]


In [ ]:
dados_btc['FILTRO'] = dados_btc.apply(processa_descricao, axis=1)


In [ ]:
dados_btc

In [ ]:
dados_btc[dados_btc['Movimentação'] == '2023-06-30']

In [ ]:
custos_totais = dados_btc.groupby(['Movimentação','FILTRO']).agg(sum_valor = ('Valor de transação', 'sum')).reset_index()
custos_totais = custos_totais[(custos_totais['FILTRO'].str.contains('COMISSÃO BTC')) | 
                              (custos_totais['FILTRO'].str.contains('TAXA DE REMUNERAÇÃO'))
#                              (custos_totais['FILTRO'].str.contains('REEMB.DIVIDENDOS')) |
#                              (custos_totais['FILTRO'].str.contains('TAXA NEGOCIACAO EMPRESTIMO'))
                             ]


In [ ]:
custos_totais

In [ ]:
custos_totais[custos_totais['Movimentação'] == '2023-06-30']

In [ ]:
# Pivot the dataframe
pivot_custos_totais = custos_totais.pivot(index='Movimentação', columns='FILTRO', values='sum_valor')

# Fill NaN values with 0
pivot_custos_totais = pivot_custos_totais.fillna(0)

# Reset the index
pivot_custos_totais.reset_index(inplace=True)

# Rename the columns
pivot_custos_totais.columns.name = None


In [ ]:
pivot_custos_totais

## Realiza a união do dataframe `dados_btc` e `pivot_custos_totais`

In [ ]:
dados_btc2 = pd.merge(dados_btc, pivot_custos_totais, how="outer", on=['Movimentação'] )
dados_btc2

In [ ]:
dados_btc2['COMISSAO_BTC_PAPEL'] = dados_btc2.apply(calcula_custos_comissao, axis=1)
dados_btc2

In [ ]:
# Define the regex pattern for extraction
pattern = r'(BR\w+)\s+TAXA NEGOCIACAO EMPRESTIMO - BTC|TAXA DE REMUNERAÇÃO - BTC\s+(BR\w+)|REEMB.DIVIDENDOS/JUROS BTC\s+(BR\w+)'

# Extract the substrings using the regex pattern and create a new column
dados_btc2['ATIVO'] = dados_btc2['Descrição'].str.extract(pattern).bfill(axis=1).iloc[:, 0].str[2:6]

#TAXA DE PERMANÊNCIA OP. ESTRUTURADAS
# Apesar de não se um custo de BTC, vou colocar em ATIVO a definir, pois é um custo que preciso inserir manualmente na Sencon
dados_btc2.loc[dados_btc2['Descrição'].str.contains("TAXA DE PERMANÊNCIA OP. ESTRUTURADAS"), 'ATIVO'] = 'A DEFINIR'



In [ ]:
dados_btc2[dados_btc2['Movimentação'] == '2023-08-07']

In [ ]:
dados_btc2[dados_btc2['Movimentação'] == '2023-06-30']

In [ ]:
# verificando se tem algum recebimento de BTC com ATIVO mapeado
# se aparecer vazio, está correto
# Vou separar esse dataset para trabalhar no futuro e remover essas linhas do dados_btc2
# dessa maneira, dados_btc2 vai ter apenas os dados de custo de quando eu estou alugando as ações
dados_btc2[(~dados_btc2['ATIVO'].isnull()) & (dados_btc2['Valor de transação'] >0)]

In [ ]:
dados_receita_btc = dados_btc2[dados_btc2['ATIVO'].isnull()]


In [ ]:
dados_receita_btc

In [ ]:
# limpeza do dados_btc2

#remove ATIVOS com NA
dados_btc2 = dados_btc2[~dados_btc2['ATIVO'].isnull()]

# remove as linhas que contém "COMISSÃO BTC"
# essa informação ficou redundante
dados_btc2 = dados_btc2[(~dados_btc2['Descrição'].str.contains('COMISSÃO BTC'))]
dados_btc2 = dados_btc2.drop(columns=['COMISSÃO BTC', 'Liquidação', 'Tipo'])



In [ ]:
dados_btc2

In [ ]:
dados_btc2['NEGOCIACAO_BTC_PAPEL'] = 0



In [ ]:
# seleciona o valor da Taxa de negociação e coloca na mesma linha que a TAXA DE REMUNERACAO
# em geral, não haverá reembolso num mesmo dia que houve 
for index, row in dados_btc2.iterrows():
    if row['FILTRO'] == 'TAXA DE REMUNERAÇÃO':
        valor = dados_btc2.loc[(dados_btc2['Movimentação']==row['Movimentação']) & 
                           (dados_btc2['ATIVO']==row['ATIVO']) &
                           (dados_btc2['FILTRO']=='TAXA NEGOCIACAO EMPRESTIMO')]
        #pega o valor da linha 0, coluna 3 (coluna 3 é a coluna do Valor de transação)
        #print(f"x{row['Movimentação']}x y{row['ATIVO']}y z{valor}z")
        #print(f"primeiro: {len(valor)}")
        if len(valor) > 0:
            valor_escalar = valor.iat[0,2]
            dados_btc2.loc[index,'NEGOCIACAO_BTC_PAPEL'] = valor_escalar

        
#        valor = dados_btc2.loc[(dados_btc2['Movimentação']==row['Movimentação']) & 
#                           (dados_btc2['ATIVO']==row['ATIVO']) &
#                           (dados_btc2['FILTRO']=='REEMB.DIVIDENDOS')]
        #pega o valor da linha 0, coluna 3 (coluna 3 é a coluna do Valor de transação)
        #print(f"segundo: {len(valor)}")
#        if len(valor) > 0:
#            valor_escalar = valor.iat[0,2]
#            dados_btc2.loc[index,'REEMB_BTC_PAPEL'] = valor_escalar
              
        #print(f"x{row['Movimentação']}x y{row['ATIVO']}y z{valor}z")

# coluna TAXA DE REMUNERAÇÃO desnecessária       
dados_btc2 = dados_btc2.drop(columns=['TAXA DE REMUNERAÇÃO'])      

# remove linhas de TAXA NEGOCIACAO EMPRESTIMO
dados_btc2 = dados_btc2[~(dados_btc2['FILTRO']=='TAXA NEGOCIACAO EMPRESTIMO')]

# adiciona coluna de TAXA DE REMUNERAÇÃO
dados_btc2['REMUNERACAO_BTC_PAPEL'] = dados_btc2.loc[dados_btc2['FILTRO'].str.contains("TAXA DE REMUNERAÇÃO"),'Valor de transação'] 

# adiciona coluna de TAXA DE PERMANENCIA
dados_btc2['TAXA_PERMANENCIA_PAPEL'] = dados_btc2.loc[dados_btc2['FILTRO'].str.contains("TAXA DE PERMANÊNCIA OP. ESTRUTURADAS"),'Valor de transação'] 

# adiciona coluna de REEMB DIVIDENDOS
dados_btc2['REEMB_DIV_PAPEL'] = dados_btc2.loc[dados_btc2['FILTRO'].str.contains("REEMB.DIVIDENDOS"),'Valor de transação'] 


# reposiciona as colunas para melhor organização
dados_btc2 = dados_btc2.reindex(['Movimentação', 'ATIVO', 'FILTRO', 'Descrição','Valor de transação','REMUNERACAO_BTC_PAPEL',
                                 'COMISSAO_BTC_PAPEL','NEGOCIACAO_BTC_PAPEL','REEMB_DIV_PAPEL' ,
                                'TAXA_PERMANENCIA_PAPEL'],axis=1)

# preenche com zero onde estiver NA
# se nao fizer isso, o CUSTO_TOTAL ficará com NA
dados_btc2.fillna(0, inplace=True)

# custo total
dados_btc2['CUSTO_TOTAL'] = dados_btc2['REMUNERACAO_BTC_PAPEL'] + \
                            dados_btc2['COMISSAO_BTC_PAPEL'] +    \
                            dados_btc2['NEGOCIACAO_BTC_PAPEL'] +  \
                            dados_btc2['REEMB_DIV_PAPEL'] +       \
                            dados_btc2['TAXA_PERMANENCIA_PAPEL'] 

# remove colunas desnecessárias
dados_btc2.drop(columns=['FILTRO','Descrição','Valor de transação'], inplace=True)




In [ ]:
# transforma todos os números em positivo
for column in dados_btc2.columns:
    if dados_btc2[column].dtypes == float:
        dados_btc2[column] = dados_btc2[column].abs()


In [ ]:
dados_btc2[dados_btc2['Movimentação'] == '2023-08-07']

In [ ]:
dados_btc2[dados_btc2['Movimentação'] == '2023-06-30']

In [ ]:
# exporta dados para o excel
dados_btc2.to_excel('custos_btc.xlsx')